The goal of this competition is to predict a Windows machine’s probability of getting infected by various families of malware, based on different properties of that machine. The telemetry data containing these properties and the machine infections was generated by combining heartbeat and threat reports collected by Microsoft's endpoint protection solution, Windows Defender.

Each row in this dataset corresponds to a machine, uniquely identified by a MachineIdentifier. HasDetections is the ground truth and indicates that Malware was detected on the machine. Using the information and labels in train.csv, you must predict the value for HasDetections for each machine in test.csv.

The sampling methodology used to create this dataset was designed to meet certain business constraints, both in regards to user privacy as well as the time period during which the machine was running. Malware detection is inherently a time-series problem, but it is made complicated by the introduction of new machines, machines that come online and offline, machines that receive patches, machines that receive new operating systems, etc. While the dataset provided here has been roughly split by time, the complications and sampling requirements mentioned above may mean you may see imperfect agreement between your cross validation, public, and private scores! Additionally, this dataset is not representative of Microsoft customers’ machines in the wild; it has been sampled to include a much larger proportion of malware machines.

In [53]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

import sklearn.ensemble as ske
from sklearn.model_selection import train_test_split
from sklearn import tree, linear_model
from sklearn.feature_selection import SelectFromModel
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics



In [54]:
# set up display area to show dataframe in jupyter qtconsole
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Columns

Unavailable or self-documenting column names are marked with an "NA".

    MachineIdentifier - Individual machine ID
    ProductName - Defender state information e.g. win8defender
    EngineVersion - Defender state information e.g. 1.1.12603.0
    AppVersion - Defender state information e.g. 4.9.10586.0
    AvSigVersion - Defender state information e.g. 1.217.1014.0
    IsBeta - Defender state information e.g. false
    RtpStateBitfield - NA
    IsSxsPassiveMode - NA
    DefaultBrowsersIdentifier - ID for the machine's default browser
    AVProductStatesIdentifier - ID for the specific configuration of a user's antivirus software
    AVProductsInstalled - NA
    AVProductsEnabled - NA
    HasTpm - True if machine has tpm
    CountryIdentifier - ID for the country the machine is located in
    CityIdentifier - ID for the city the machine is located in
    OrganizationIdentifier - ID for the organization the machine belongs in, organization ID is mapped to both specific companies and broad industries
    GeoNameIdentifier - ID for the geographic region a machine is located in
    LocaleEnglishNameIdentifier - English name of Locale ID of the current user
    Platform - Calculates platform name (of OS related properties and processor property)
    Processor - This is the process architecture of the installed operating system
    OsVer - Version of the current operating system
    OsBuild - Build of the current operating system
    OsSuite - Product suite mask for the current operating system.
    OsPlatformSubRelease - Returns the OS Platform sub-release (Windows Vista, Windows 7, Windows 8, TH1, TH2)
    OsBuildLab - Build lab that generated the current OS. Example: 9600.17630.amd64fre.winblue_r7.150109-2022
    SkuEdition - The goal of this feature is to use the Product Type defined in the MSDN to map to a 'SKU-Edition' name that is useful in population reporting. The valid Product Type are defined in %sdxroot%\data\windowseditions.xml. This API has been used since Vista and Server 2008, so there are many Product Types that do not apply to Windows 10. The 'SKU-Edition' is a string value that is in one of three classes of results. The design must hand each class.
    IsProtected - This is a calculated field derived from the Spynet Report's AV Products field. Returns: a. TRUE if there is at least one active and up-to-date antivirus product running on this machine. b. FALSE if there is no active AV product on this machine, or if the AV is active, but is not receiving the latest updates. c. null if there are no Anti Virus Products in the report. Returns: Whether a machine is protected.
    AutoSampleOptIn - This is the SubmitSamplesConsent value passed in from the service, available on CAMP 9+
    PuaMode - Pua Enabled mode from the service
    SMode - This field is set to true when the device is known to be in 'S Mode', as in, Windows 10 S mode, where only Microsoft Store apps can be installed
    IeVerIdentifier - NA
    SmartScreen - This is the SmartScreen enabled string value from registry. This is obtained by checking in order, HKLM\SOFTWARE\Policies\Microsoft\Windows\System\SmartScreenEnabled and HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\SmartScreenEnabled. If the value exists but is blank, the value "ExistsNotSet" is sent in telemetry.
    Firewall - This attribute is true (1) for Windows 8.1 and above if windows firewall is enabled, as reported by the service.
    UacLuaenable - This attribute reports whether or not the "administrator in Admin Approval Mode" user type is disabled or enabled in UAC. The value reported is obtained by reading the regkey HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System\EnableLUA.
    Census_MDC2FormFactor - A grouping based on a combination of Device Census level hardware characteristics. The logic used to define Form Factor is rooted in business and industry standards and aligns with how people think about their device. (Examples: Smartphone, Small Tablet, All in One, Convertible...)
    Census_DeviceFamily - AKA DeviceClass. Indicates the type of device that an edition of the OS is intended for. Example values: Windows.Desktop, Windows.Mobile, and iOS.Phone
    Census_OEMNameIdentifier - NA
    Census_OEMModelIdentifier - NA
    Census_ProcessorCoreCount - Number of logical cores in the processor
    Census_ProcessorManufacturerIdentifier - NA
    Census_ProcessorModelIdentifier - NA
    Census_ProcessorClass - A classification of processors into high/medium/low. Initially used for Pricing Level SKU. No longer maintained and updated
    Census_PrimaryDiskTotalCapacity - Amount of disk space on primary disk of the machine in MB
    Census_PrimaryDiskTypeName - Friendly name of Primary Disk Type - HDD or SSD
    Census_SystemVolumeTotalCapacity - The size of the partition that the System volume is installed on in MB
    Census_HasOpticalDiskDrive - True indicates that the machine has an optical disk drive (CD/DVD)
    Census_TotalPhysicalRAM - Retrieves the physical RAM in MB
    Census_ChassisTypeName - Retrieves a numeric representation of what type of chassis the machine has. A value of 0 means xx
    Census_InternalPrimaryDiagonalDisplaySizeInInches - Retrieves the physical diagonal length in inches of the primary display
    Census_InternalPrimaryDisplayResolutionHorizontal - Retrieves the number of pixels in the horizontal direction of the internal display.
    Census_InternalPrimaryDisplayResolutionVertical - Retrieves the number of pixels in the vertical direction of the internal display
    Census_PowerPlatformRoleName - Indicates the OEM preferred power management profile. This value helps identify the basic form factor of the device
    Census_InternalBatteryType - NA
    Census_InternalBatteryNumberOfCharges - NA
    Census_OSVersion - Numeric OS version Example - 10.0.10130.0
    Census_OSArchitecture - Architecture on which the OS is based. Derived from OSVersionFull. Example - amd64
    Census_OSBranch - Branch of the OS extracted from the OsVersionFull. Example - OsBranch = fbl_partner_eeap where OsVersion = 6.4.9813.0.amd64fre.fbl_partner_eeap.140810-0005
    Census_OSBuildNumber - OS Build number extracted from the OsVersionFull. Example - OsBuildNumber = 10512 or 10240
    Census_OSBuildRevision - OS Build revision extracted from the OsVersionFull. Example - OsBuildRevision = 1000 or 16458
    Census_OSEdition - Edition of the current OS. Sourced from HKLM\Software\Microsoft\Windows NT\CurrentVersion@EditionID in registry. Example: Enterprise
    Census_OSSkuName - OS edition friendly name (currently Windows only)
    Census_OSInstallTypeName - Friendly description of what install was used on the machine i.e. clean
    Census_OSInstallLanguageIdentifier - NA
    Census_OSUILocaleIdentifier - NA
    Census_OSWUAutoUpdateOptionsName - Friendly name of the WindowsUpdate auto-update settings on the machine.
    Census_IsPortableOperatingSystem - Indicates whether OS is booted up and running via Windows-To-Go on a USB stick.
    Census_GenuineStateName - Friendly name of OSGenuineStateID. 0 = Genuine
    Census_ActivationChannel - Retail license key or Volume license key for a machine.
    Census_IsFlightingInternal - NA
    Census_IsFlightsDisabled - Indicates if the machine is participating in flighting.
    Census_FlightRing - The ring that the device user would like to receive flights for. This might be different from the ring of the OS which is currently installed if the user changes the ring after getting a flight from a different ring.
    Census_ThresholdOptIn - NA
    Census_FirmwareManufacturerIdentifier - NA
    Census_FirmwareVersionIdentifier - NA
    Census_IsSecureBootEnabled - Indicates if Secure Boot mode is enabled.
    Census_IsWIMBootEnabled - NA
    Census_IsVirtualDevice - Identifies a Virtual Machine (machine learning model)
    Census_IsTouchEnabled - Is this a touch device ?
    Census_IsPenCapable - Is the device capable of pen input ?
    Census_IsAlwaysOnAlwaysConnectedCapable - Retreives information about whether the battery enables the device to be AlwaysOnAlwaysConnected .
    Wdft_IsGamer - Indicates whether the device is a gamer device or not based on its hardware combination.
    Wdft_RegionIdentifier - NA


In [55]:
# We need to explicitly specify data types when reading csv, otherwise it is very memory consuming
# and we will get the warning "Specify dtype option on import or set low_memory=False"
# So, we will manually defined the data types

# P.S. I have loaded the sample data and exported train_data.dtypes
# these are the data types for fast loading

datatypes = {
    'ProductName': str,
    'EngineVersion': str,
    'AppVersion': str,
    'AvSigVersion': str,
    'IsBeta': np.int8,
    'RtpStateBitfield': str,
    'IsSxsPassiveMode': np.int8,
    'DefaultBrowsersIdentifier': str,
    'AVProductStatesIdentifier': str,
    'AVProductsInstalled': str,
    'AVProductsEnabled': str,
    'HasTpm': np.int8,
    'CountryIdentifier': str,
    'CityIdentifier': str,
    'OrganizationIdentifier': str,
    'GeoNameIdentifier': str,
    'LocaleEnglishNameIdentifier': str,
    'Platform': str,
    'Processor': str,
    'OsVer': str,
    'OsBuild': str,
    'OsSuite': str,
    'OsPlatformSubRelease': str,
    'OsBuildLab': str,
    'SkuEdition': str,
    'IsProtected': str,
    'AutoSampleOptIn': np.int8,
    'PuaMode': str,
    'SMode': str,
    'IeVerIdentifier': str,
    'SmartScreen': str,
    'Firewall': str,
    'UacLuaenable': str,
    'Census_MDC2FormFactor': str,
    'Census_DeviceFamily': str,
    'Census_OEMNameIdentifier': str,
    'Census_OEMModelIdentifier': str, 
    'Census_ProcessorCoreCount': str,
    'Census_ProcessorManufacturerIdentifier': str,
    'Census_ProcessorModelIdentifier': str,
    'Census_ProcessorClass': str,
    'Census_PrimaryDiskTotalCapacity': np.float64,
    'Census_PrimaryDiskTypeName': str,
    'Census_SystemVolumeTotalCapacity': np.float64,
    'Census_HasOpticalDiskDrive': np.int8,
    'Census_TotalPhysicalRAM': np.float64,
    'Census_ChassisTypeName': str,
    'Census_InternalPrimaryDiagonalDisplaySizeInInches': str,
    'Census_InternalPrimaryDisplayResolutionHorizontal': str,
    'Census_InternalPrimaryDisplayResolutionVertical': str,
    'Census_PowerPlatformRoleName': str,
    'Census_InternalBatteryType': str,
    'Census_InternalBatteryNumberOfCharges': str,
    'Census_OSVersion': str,
    'Census_OSArchitecture': str,
    'Census_OSBranch': str,
    'Census_OSBuildNumber': str,
    'Census_OSBuildRevision': str,
    'Census_OSEdition': str,
    'Census_OSSkuName': str,
    'Census_OSInstallTypeName': str,
    'Census_OSInstallLanguageIdentifier': str,
    'Census_OSUILocaleIdentifier': str,
    'Census_OSWUAutoUpdateOptionsName': str,
    'Census_IsPortableOperatingSystem': np.int8,
    'Census_GenuineStateName': str,
    'Census_ActivationChannel': str,
    'Census_IsFlightingInternal': str,
    'Census_IsFlightsDisabled': str,
    'Census_FlightRing': str,
    'Census_ThresholdOptIn': str,
    'Census_FirmwareManufacturerIdentifier': str,
    'Census_FirmwareVersionIdentifier': str,
    'Census_IsSecureBootEnabled': np.int8,
    'Census_IsWIMBootEnabled': str,
    'Census_IsVirtualDevice': str,
    'Census_IsTouchEnabled': np.int8,
    'Census_IsPenCapable': np.int8,
    'Census_IsAlwaysOnAlwaysConnectedCapable': str,
    'Wdft_IsGamer': str,
    'Wdft_RegionIdentifier': str,
    'HasDetections': np.int8
}

full_features = pd.read_csv("./csv/train.csv", dtype=datatypes, index_col="MachineIdentifier")
#full_features = pd.read_csv("./csv/train.csv", dtype=datatypes, nrows=200000, index_col="MachineIdentifier")
full_labels = full_features["HasDetections"]

# Dropping labels ["HasDetections"] from training dataset
full_features = full_features.drop(["HasDetections"], axis=1)

In [56]:
#test_features = pd.read_csv("./csv/test.csv", dtype=datatypes, index_col="MachineIdentifier")

In [57]:
#full_features = pd.concat([full_features, test_features])

In [58]:
print (full_features.shape)

(8921483, 81)


In [59]:
# Checking the columns with the most NULL values
print((full_features.isnull().sum()).sort_values(ascending=False).head(20))

PuaMode                                  8919174
Census_ProcessorClass                    8884852
DefaultBrowsersIdentifier                8488045
Census_IsFlightingInternal               7408759
Census_InternalBatteryType               6338429
Census_ThresholdOptIn                    5667325
Census_IsWIMBootEnabled                  5659703
SmartScreen                              3177011
OrganizationIdentifier                   2751518
SMode                                     537759
CityIdentifier                            325409
Wdft_IsGamer                              303451
Wdft_RegionIdentifier                     303451
Census_InternalBatteryNumberOfCharges     268755
Census_FirmwareManufacturerIdentifier     183257
Census_IsFlightsDisabled                  160523
Census_FirmwareVersionIdentifier          160133
Census_OEMModelIdentifier                 102233
Census_OEMNameIdentifier                   95478
Firewall                                   91350
dtype: int64


In [60]:
full_features['PuaMode'].unique()

array([nan, 'on', 'audit'], dtype=object)

In [61]:
full_features['Census_IsFlightingInternal'].unique()

array([nan, '0', '1'], dtype=object)

In [62]:
full_features['Census_InternalBatteryType'].unique()

array([nan, 'lion', 'li-i', '#', 'lip', 'liio', 'vbox', 'li p', 'real',
       'unkn', 'pbac', 'li', 'bq20', 'nimh', '\x04lio', 'lgi0', 'lhp0',
       'ithi', 'batt', 'lipp', 'lipo', '4cel', 'ram', 'lit', 'a140',
       'bad', 'asmb', 'virt', 'ca48', '4ion', 'd', 'a132', 'ÿÿÿÿ', 'cl53',
       'lio', 'li-l', '÷ÿóö', 'í\x03-i', '0x0b', 'lgs0', '3ion', 'ots0',
       'lai0', 'lilo', 'pa50', 'h4°s', '5nm1', 'li-p', 'lhpo', '0ts0',
       'pad0', 'sail', 'p-sn', 'icp3', 'a130', '2337', '\x1f˙˙˙', 'lgl0',
       'l\x15', '@i\uf8f5\uf8f5', 'li\x90o', '4lio', 'lp', 'li?',
       '\x04ion', 'pbso', 'a138', 'li-h', '6ion', '3500', 'h00j',
       'li\x10', 'sams', '\x03ip', '8', '#TAB#', 'l\x06&#TAB#', 'liÿÿ',
       'lÿÿÿ'], dtype=object)

In [63]:
full_features['Census_ThresholdOptIn'].unique()

array([nan, '0', '1'], dtype=object)

In [64]:
full_features['Census_IsWIMBootEnabled'].unique()

array([nan, '0', '1'], dtype=object)

In [65]:
full_features['SMode'].unique()

array(['0', nan, '1'], dtype=object)

In [66]:
full_features['OrganizationIdentifier'].unique()

array(['18', nan, '27', '46', '11', '14', '37', '10', '50', '49', '33',
       '8', '48', '36', '31', '4', '1', '28', '3', '52', '32', '51', '5',
       '2', '47', '44', '16', '40', '20', '22', '29', '26', '21', '39',
       '6', '19', '7', '30', '42', '43', '41', '15', '45', '25', '35',
       '23', '38', '12', '17', '34'], dtype=object)

In [67]:
full_features['Wdft_IsGamer'].unique()

array(['0', '1', nan], dtype=object)

In [68]:
full_features['Wdft_RegionIdentifier'].unique()

array(['10', '8', '3', '1', '15', '7', '11', '2', '12', '4', '13', nan,
       '6', '9', '5', '14'], dtype=object)

In [69]:
full_features['CityIdentifier'].unique()

array(['128035', '1482', '153579', ..., '47472', '147921', '97837'],
      dtype=object)

In [70]:
full_features['Census_InternalBatteryNumberOfCharges'].unique()

array(['4294967295', '1', '0', ..., '27736', '26424', '16807'],
      dtype=object)

In [71]:
# Cleaning up some data

# PuaMode - Potentially Unwanted Applications, if NA, then it is disabled
# Census_ProcessorClass - According to the description - "No longer maintained and updated"
# DefaultBrowsersIdentifier - Almost all values are empty. Therefore we will drop this column
# Census_IsFlightingInternal - whether this is internal or "external" testing ring. Column mostly unused. Will have to drop it
# Census_InternalBatteryType - comtains mostly garbage. Besides, it should not be relevant to attack surface.
# Census_ThresholdOptIn - also mostly unused. Googled it and Threshold was used in first versions of Windows 10. Looks like unused now
# Census_IsWIMBootEnabled - Is it possible to boot from Windows Image? Not relevant to identification of the attacks when 70% of data is emtpy
# SmartScreen - Whether smart screen in explorer is enabled. Should be important. "ExistsNotSet" when null, according to the description
# SMode - Quite relevant field. Will be keeping it
# OrganizationIdentifier - Attacks by organizations should be analyzed. If not filled, will assign "0". 
# Census_InternalBatteryNumberOfCharges - Not relevant. Will drop this column in order not to overtrain
# Census_OSSkuName -  OS edition friendly name (currently Windows only). - Can be removed. Duplicate field
# Census_ChassisTypeName - Census_MDC2FormFactor gives better information. Let's remove this field

full_features['PuaMode'] = full_features['PuaMode'].fillna('off')
full_features['SmartScreen'] = full_features['SmartScreen'].fillna('ExistsNotSet')
full_features['SMode'] = full_features['SMode'].fillna('0').astype('int8')
full_features['OrganizationIdentifier'] = full_features['OrganizationIdentifier'].fillna('0').astype('int32')
full_features['Wdft_IsGamer'] = full_features['Wdft_IsGamer'].fillna('0').astype('int8')
full_features['Wdft_RegionIdentifier'] = full_features['Wdft_RegionIdentifier'].fillna('0').astype('int32')
full_features['CityIdentifier'] = full_features['CityIdentifier'].fillna('0').astype('int32')

full_features = full_features.drop([
    'Census_ProcessorClass',
    'DefaultBrowsersIdentifier',
    'Census_IsFlightingInternal',
    'Census_InternalBatteryType',
    'Census_ThresholdOptIn',
    'Census_IsWIMBootEnabled',
    'Census_InternalBatteryNumberOfCharges',
    'Census_OSSkuName',
    'Census_ChassisTypeName'], axis=1)

In [72]:
# Removing duplicate fields
# Census_OSBranch, Census_OSBuildNumber, Census_OSBuildRevision, Census_OSArchitecture, OsBuild, ProductName

full_features = full_features.drop([
    'Census_OSBranch',
    'Census_OSBuildNumber',
    'Census_OSBuildRevision',
    'Census_OSArchitecture',
    'OsBuild',
    'ProductName'], axis=1)

In [73]:
# Now let us check the string columns

string_columns = []

for colname in full_features.dtypes.keys():
    if full_features[colname].dtypes.name == "object":
        string_columns.append(colname)
        
string_columns

['EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'RtpStateBitfield',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'AVProductsEnabled',
 'CountryIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'Platform',
 'Processor',
 'OsVer',
 'OsSuite',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'IsProtected',
 'PuaMode',
 'IeVerIdentifier',
 'SmartScreen',
 'Firewall',
 'UacLuaenable',
 'Census_MDC2FormFactor',
 'Census_DeviceFamily',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_PrimaryDiskTypeName',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_PowerPlatformRoleName',
 'Census_OSVersion',
 'Census_OSEdition',
 'Census_OSInstallTypeName',
 'Census_OSInstallLanguageIdentifier',
 'Census_OSUILocaleIdentifi

In [74]:
full_features[string_columns].head(10)

,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,CountryIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,PuaMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_OSVersion,Census_OSEdition,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightsDisabled,Census_FlightRing,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsVirtualDevice,Census_IsAlwaysOnAlwaysConnectedCapable
MachineIdentifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0000028988387b115f69f31a3bf04f09,1.1.15100.1,4.18.1807.18075,1.273.1735.0,7,53447,1,1,29,35,171,windows10,x64,10.0.0.0,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1,off,137,ExistsNotSet,1,1,Desktop,Windows.Desktop,2668,9124,4,5,2341,HDD,18.9,1440,900,Desktop,10.0.17134.165,Professional,UUPUpgrade,26,119,UNKNOWN,IS_GENUINE,Retail,0,Retail,628,36144,0,0
000007535c3f730efa9ea0b7ef1bd645,1.1.14600.4,4.13.17134.1,1.263.48.0,7,53447,1,1,93,119,64,windows10,x64,10.0.0.0,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1,off,137,ExistsNotSet,1,1,Notebook,Windows.Desktop,2668,91656,4,5,2405,HDD,13.9,1366,768,Mobile,10.0.17134.1,Professional,IBSClean,8,31,UNKNOWN,OFFLINE,Retail,0,NOT_SET,628,57858,0,0
000007905a28d863f6d0d597892cd692,1.1.15100.1,4.18.1807.18075,1.273.1341.0,7,53447,1,1,86,64,49,windows10,x64,10.0.0.0,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1,off,137,RequireAdmin,1,1,Desktop,Windows.Desktop,4909,317701,4,5,1972,SSD,21.5,1920,1080,Desktop,10.0.17134.165,Core,UUPUpgrade,7,30,FullAuto,IS_GENUINE,OEM:NONSLP,0,Retail,142,52682,0,0
00000b11598a75ea8ba1beea8459149f,1.1.15100.1,4.18.1807.18075,1.273.1527.0,7,53447,1,1,88,117,115,windows10,x64,10.0.0.0,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1,off,137,ExistsNotSet,1,1,Desktop,Windows.Desktop,1443,275890,4,5,2273,UNKNOWN,18.5,1366,768,Desktop,10.0.17134.228,Professional,UUPUpgrade,17,64,FullAuto,IS_GENUINE,OEM:NONSLP,0,Retail,355,20050,0,0
000014a5f00daa18e76b81417eeb99fc,1.1.15100.1,4.18.1807.18075,1.273.1379.0,7,53447,1,1,18,277,75,windows10,x64,10.0.0.0,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1,off,137,RequireAdmin,1,1,Notebook,Windows.Desktop,1443,331929,4,5,2500,HDD,14,1366,768,Mobile,10.0.17134.191,Core,Update,8,31,FullAuto,IS_GENUINE,Retail,0,Retail,355,19844,0,0
000016191b897145d069102325cab760,1.1.15100.1,4.18.1807.18075,1.273.1094.0,7,53447,1,1,97,126,124,windows10,x64,10.0.0.0,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1,off,137,RequireAdmin,1,1,Desktop,Windows.Desktop,3799,340727,2,5,4324,SSD,21.5,1920,1080,Desktop,10.0.17134.165,Professional,UUPUpgrade,18,72,FullAuto,IS_GENUINE,Retail,0,Retail,93,51039,0,0
0000161e8abf8d8b89c5ab8787fd712b,1.1.15100.1,4.18.1807.18075,1.273.845.0,7,43927,2,1,78,89,88,windows10,x64,10.0.0.0,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1,off,137,ExistsNotSet,1,1,Notebook,Windows.Desktop,3799,207404,2,1,657,HDD,17.2,1600,900,Mobile,10.0.17134.165,Core,IBSClean,14,49,FullAuto,IS_GENUINE,Retail,0,Retail,556,63175,0,0
000019515bc8f95851aff6de873405e8,1.1.15100.1,4.18.1807.18075,1.273.1393.0,7,53447,1,1,97,126,124,windows10,x64,10.0.0.0,768,rs1,14393.0.amd64fre.rs1_release.160715-1616,Home,1,off,94,RequireAdmin,1,1,Notebook,Windows.Desktop,5682,338896,2,5,3381,HDD,15.5,1366,768,Mobile,10.0.14393.0,Core,Upgrade,18,72,FullAu

At first glance at the data, it becomes obvious, that the stings are either classifiers, or versions that contain 4 classifiers in them. So. in order to use the algorithms that support only numeric values we will convert classifiers like "ProductName" to integer range and the fields like AppVersion

In [75]:
def df_replacevalues(df, colname, oldvalues, newvalues):
    # First, we need to get the most frequent value of the column
    topvalue = df[colname].value_counts().idxmax()
    
    # Replace NaN values with the popular value
    df[colname].fillna(topvalue, inplace=True)
    
    # We need to make sure no other value than oldvalues exists
    indexes = df[~df[colname].isin(oldvalues)].index
    
    # If the "Garbage" values are more than 1%, then raise an error
    if len(indexes) > len(df) / 100:
        raise Exception("Not all neccessary values are present in oldvalues array")
    
    # Replace "Garbage" with the top value
    df.loc[indexes,[colname]] = topvalue
    
    print ("Previous values", df[colname].unique())
    df[colname] = pd.to_numeric(df[colname].replace(oldvalues, newvalues), errors='raise', downcast='integer')
    print ("New values", df[colname].unique())
    
#full_features["Platform"].unique()
#full_features["Platform"].value_counts()
#full_features[~full_features["ProductName"].isin(['win8defender', 'mse'])].index

In [76]:
colname = "Platform"
oldvalues = ['windows10','windows7','windows8','windows2016']
newvalues = [10,7,8,2016]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['windows10' 'windows7' 'windows8' 'windows2016']
New values [  10    7    8 2016]


In [77]:
colname = "Processor"
oldvalues = ['x64','arm64','x86']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['x64' 'arm64' 'x86']
New values [1 2 3]


In [78]:
colname = "OsPlatformSubRelease"
oldvalues = ['rs4','rs1','rs3','windows7','windows8.1','th1','rs2','th2','prers5']
newvalues = [504,501,503,507,508,201,502,202,405]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['rs4' 'rs1' 'rs3' 'windows7' 'windows8.1' 'th1' 'rs2' 'th2' 'prers5']
New values [504 501 503 507 508 201 502 202 405]


In [79]:
colname = "SkuEdition"
oldvalues = ['Pro','Home','Invalid','Enterprise LTSB','Enterprise','Education','Cloud','Server']
newvalues = [55,52,0,71,70,20,90,80]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['Pro' 'Home' 'Invalid' 'Enterprise LTSB' 'Enterprise' 'Education' 'Cloud'
 'Server']
New values [55 52  0 71 70 20 90 80]


In [80]:
colname = "SmartScreen"
oldvalues = ['Off','off','OFF','On','on','Warn','Prompt','ExistsNotSet','Block','RequireAdmin']
newvalues = [0,0,0,1,1,2,3,4,5,6]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['ExistsNotSet' 'RequireAdmin' 'Off' 'Warn' 'Prompt' 'Block' 'off' 'On'
 'on' 'OFF']
New values [4 6 0 2 3 5 1]


In [81]:
colname = "Census_MDC2FormFactor"
oldvalues = ['Desktop','Notebook','Detachable','PCOther','AllInOne','Convertible','SmallTablet','LargeTablet','SmallServer','LargeServer','MediumServer','ServerOther','IoTOther']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['Desktop' 'Notebook' 'Detachable' 'PCOther' 'AllInOne' 'Convertible'
 'SmallTablet' 'LargeTablet' 'SmallServer' 'LargeServer' 'MediumServer'
 'ServerOther' 'IoTOther']
New values [ 1  2  3  4  5  6  7  8  9 10 11 12 13]


In [82]:
# Census_DeviceFamily ['Windows.Desktop' 'Windows.Server' 'Windows']

colname = "Census_DeviceFamily"
oldvalues = ['Windows.Desktop','Windows.Server','Windows']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['Windows.Desktop' 'Windows.Server' 'Windows']
New values [1 2 3]


In [83]:
# Census_PrimaryDiskTypeName ['HDD' 'SSD' 'UNKNOWN' 'Unspecified' nan]

colname = "Census_PrimaryDiskTypeName"
oldvalues = ['HDD','SSD','UNKNOWN','Unspecified']
newvalues = [1,2,3,3]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['HDD' 'SSD' 'UNKNOWN' 'Unspecified']
New values [1 2 3]


In [84]:
# Census_PowerPlatformRoleName Index(['Mobile', 'Desktop', 'Slate', 'Workstation', 'SOHOServer', 'UNKNOWN', 'EnterpriseServer', 'AppliancePC', 'PerformanceServer', 'Unspecified']

colname = "Census_PowerPlatformRoleName"
full_features[colname] = full_features[colname].fillna('UNKNOWN')
oldvalues = ['Mobile', 'Desktop', 'Slate', 'Workstation', 'SOHOServer', 'UNKNOWN', 'EnterpriseServer', 'AppliancePC', 'PerformanceServer', 'Unspecified']
newvalues = [1,2,3,2,4,0,5,6,7,0]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['Desktop' 'Mobile' 'Slate' 'Workstation' 'SOHOServer' 'UNKNOWN'
 'AppliancePC' 'EnterpriseServer' 'PerformanceServer' 'Unspecified']
New values [2 1 3 4 0 6 5 7]


In [85]:
# Census_OSSkuName Index(['CORE', 'PROFESSIONAL', 'CORE_SINGLELANGUAGE', 'CORE_COUNTRYSPECIFIC', 'EDUCATION', 'ENTERPRISE', 'PROFESSIONAL_N', 'ENTERPRISE_S', 'STANDARD_SERVER', 'CLOUD', 'CORE_N', 'STANDARD_EVALUATION_SERVER', 'EDUCATION_N', 'ENTERPRISE_S_N', 'DATACENTER_EVALUATION_SERVER', 'SB_SOLUTION_SERVER', 'ENTERPRISE_N', 'PRO_WORKSTATION', 'UNLICENSED', 'DATACENTER_SERVER', 'PRO_WORKSTATION_N', 'CLOUDN', 'PRO_CHINA', 'SERVERRDSH', 'ULTIMATE', 'PRO_FOR_EDUCATION', 'PRO_SINGLE_LANGUAGE', 'UNDEFINED', 'STARTER', 'ENTERPRISEG'], dtype='object')

#colname = "Census_OSSkuName"
#oldvalues = ['CORE', 'PROFESSIONAL', 'CORE_SINGLELANGUAGE', 'CORE_COUNTRYSPECIFIC', 'EDUCATION', 'ENTERPRISE', 'PROFESSIONAL_N', 'ENTERPRISE_S', 'STANDARD_SERVER', 'CLOUD', 'CORE_N', 'STANDARD_EVALUATION_SERVER', 'EDUCATION_N', 'ENTERPRISE_S_N', 'DATACENTER_EVALUATION_SERVER', 'SB_SOLUTION_SERVER', 'ENTERPRISE_N', 'PRO_WORKSTATION', 'UNLICENSED']
#newvalues = [i+1 for i in range(len(oldvalues))]

#df_replacevalues(full_features, colname, oldvalues, newvalues)

In [86]:
# Census_OSInstallTypeName Index(['UUPUpgrade', 'IBSClean', 'Update', 'Upgrade', 'Other', 'Reset', 'Refresh', 'Clean', 'CleanPCRefresh'], dtype='object')

colname = "Census_OSInstallTypeName"
oldvalues = ['UUPUpgrade', 'IBSClean', 'Update', 'Upgrade', 'Other', 'Reset', 'Refresh', 'Clean', 'CleanPCRefresh']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['UUPUpgrade' 'IBSClean' 'Update' 'Upgrade' 'Other' 'Clean' 'Reset'
 'Refresh' 'CleanPCRefresh']
New values [1 2 3 4 5 8 6 7 9]


In [87]:
# Census_OSWUAutoUpdateOptionsName Index(['FullAuto', 'UNKNOWN', 'Notify', 'AutoInstallAndRebootAtMaintenanceTime', 'Off', 'DownloadNotify'], dtype='object')

colname = "Census_OSWUAutoUpdateOptionsName"
oldvalues = ['FullAuto', 'UNKNOWN', 'Notify', 'AutoInstallAndRebootAtMaintenanceTime', 'Off', 'DownloadNotify']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['UNKNOWN' 'FullAuto' 'Notify' 'AutoInstallAndRebootAtMaintenanceTime'
 'Off' 'DownloadNotify']
New values [2 1 3 4 5 6]


In [88]:
# Census_GenuineStateName Index(['IS_GENUINE', 'INVALID_LICENSE', 'OFFLINE', 'UNKNOWN', 'TAMPERED'], dtype='object')

colname = "Census_GenuineStateName"
oldvalues = ['IS_GENUINE', 'INVALID_LICENSE', 'OFFLINE', 'UNKNOWN', 'TAMPERED']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['IS_GENUINE' 'OFFLINE' 'INVALID_LICENSE' 'UNKNOWN' 'TAMPERED']
New values [1 3 2 4 5]


In [89]:
# Census_ActivationChannel Index(['Retail', 'OEM:DM', 'Volume:GVLK', 'OEM:NONSLP', 'Volume:MAK', 'Retail:TB:Eval'], dtype='object')

colname = "Census_ActivationChannel"
oldvalues = ['Retail', 'OEM:DM', 'Volume:GVLK', 'OEM:NONSLP', 'Volume:MAK', 'Retail:TB:Eval']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['Retail' 'OEM:NONSLP' 'OEM:DM' 'Volume:GVLK' 'Volume:MAK'
 'Retail:TB:Eval']
New values [1 4 2 3 5 6]


In [90]:
# Census_FlightRing Index(['Retail', 'NOT_SET', 'Unknown', 'WIS', 'WIF', 'RP', 'Disabled', 'OSG', 'Canary', 'Invalid', 'CBCanary'], dtype='object')

colname = "Census_FlightRing"
oldvalues = ['Retail', 'NOT_SET', 'Unknown', 'WIS', 'WIF', 'RP', 'Disabled', 'OSG', 'Canary', 'Invalid', 'CBCanary']
newvalues = [1,2,0,3,4,5,0,0,0,0,0]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['Retail' 'NOT_SET' 'Unknown' 'Disabled' 'RP' 'WIS' 'WIF' 'OSG' 'Canary'
 'Invalid']
New values [1 2 0 5 3 4]


In [91]:
# PuaMode Index(['off', 'on', 'audit'], dtype='object')

colname = "PuaMode"
oldvalues = ['off', 'on', 'audit']
newvalues = [0,1,2]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['off' 'on' 'audit']
New values [0 1 2]


In [92]:
# Census_OSEdition

colname = "Census_OSEdition"
oldvalues = ['Core','Professional','CoreSingleLanguage','CoreCountrySpecific','ProfessionalEducation','Education',
             'Enterprise','ProfessionalN','EnterpriseS','ServerStandard','Cloud','CoreN']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['Professional' 'Core' 'CoreSingleLanguage' 'EnterpriseS'
 'CoreCountrySpecific' 'ProfessionalEducation' 'Enterprise' 'Education'
 'Cloud' 'ProfessionalN' 'ServerStandard' 'CoreN']
New values [ 2  1  3  9  4  5  7  6 11  8 10 12]


In [93]:
# Now let us check the string columns again

string_columns = []

for colname in full_features.dtypes.keys():
    if full_features[colname].dtypes.name == "object":
        string_columns.append(colname)
        
string_columns

['EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'RtpStateBitfield',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'AVProductsEnabled',
 'CountryIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'OsVer',
 'OsSuite',
 'OsBuildLab',
 'IsProtected',
 'IeVerIdentifier',
 'Firewall',
 'UacLuaenable',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_OSVersion',
 'Census_OSInstallLanguageIdentifier',
 'Census_OSUILocaleIdentifier',
 'Census_IsFlightsDisabled',
 'Census_FirmwareManufacturerIdentifier',
 'Census_FirmwareVersionIdentifier',
 'Census_IsVirtualDevice',
 'Census_IsAlwaysOnAlwaysConnectedCapable']

In [94]:
full_features[string_columns].head(10)

,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,CountryIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsVer,OsSuite,OsBuildLab,IsProtected,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_OSVersion,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsFlightsDisabled,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsVirtualDevice,Census_IsAlwaysOnAlwaysConnectedCapable
MachineIdentifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0000028988387b115f69f31a3bf04f09,1.1.15100.1,4.18.1807.18075,1.273.1735.0,7,53447,1,1,29,35,171,10.0.0.0,256,17134.1.amd64fre.rs4_release.180410-1804,1,137,1,1,2668,9124,4,5,2341,18.9,1440,900,10.0.17134.165,26,119,0,628,36144,0,0
000007535c3f730efa9ea0b7ef1bd645,1.1.14600.4,4.13.17134.1,1.263.48.0,7,53447,1,1,93,119,64,10.0.0.0,256,17134.1.amd64fre.rs4_release.180410-1804,1,137,1,1,2668,91656,4,5,2405,13.9,1366,768,10.0.17134.1,8,31,0,628,57858,0,0
000007905a28d863f6d0d597892cd692,1.1.15100.1,4.18.1807.18075,1.273.1341.0,7,53447,1,1,86,64,49,10.0.0.0,768,17134.1.amd64fre.rs4_release.180410-1804,1,137,1,1,4909,317701,4,5,1972,21.5,1920,1080,10.0.17134.165,7,30,0,142,52682,0,0
00000b11598a75ea8ba1beea8459149f,1.1.15100.1,4.18.1807.18075,1.273.1527.0,7,53447,1,1,88,117,115,10.0.0.0,256,17134.1.amd64fre.rs4_release.180410-1804,1,137,1,1,1443,275890,4,5,2273,18.5,1366,768,10.0.17134.228,17,64,0,355,20050,0,0
000014a5f00daa18e76b81417eeb99fc,1.1.15100.1,4.18.1807.18075,1.273.1379.0,7,53447,1,1,18,277,75,10.0.0.0,768,17134.1.amd64fre.rs4_release.180410-1804,1,137,1,1,1443,331929,4,5,2500,14,1366,768,10.0.17134.191,8,31,0,355,19844,0,0
000016191b897145d069102325cab760,1.1.15100.1,4.18.1807.18075,1.273.1094.0,7,53447,1,1,97,126,124,10.0.0.0,256,17134.1.amd64fre.rs4_release.180410-1804,1,137,1,1,3799,340727,2,5,4324,21.5,1920,1080,10.0.17134.165,18,72,0,93,51039,0,0
0000161e8abf8d8b89c5ab8787fd712b,1.1.15100.1,4.18.1807.18075,1.273.845.0,7,43927,2,1,78,89,88,10.0.0.0,768,17134.1.amd64fre.rs4_release.180410-1804,1,137,1,1,3799,207404,2,1,657,17.2,1600,900,10.0.17134.165,14,49,0,556,63175,0,0
000019515bc8f95851aff6de873405e8,1.1.15100.1,4.18.1807.18075,1.273.1393.0,7,53447,1,1,97,126,124,10.0.0.0,768,14393.0.amd64fre.rs1_release.160715-1616,1,94,1,1,5682,338896,2,5,3381,15.5,1366,768,10.0.14393.0,18,72,0,512,63122,0,0
00001a027a0ab970c408182df8484fce,1.1.15200.1,4.18.1807.18075,1.275.988.0,7,53447,1,1,164,205,172,10.0.0.0,256,17134.1.amd64fre.rs4_release.180410-1804,1,137,1,1,2206,240688,4,5,2836,15.6,1920,1080,10.0.17134.254,27,120,0,500,15510,0,0


In [95]:
# Now we need to process the columns that contain version numbers
# We will split them in 4-5 different columns

versions = ['EngineVersion','AppVersion','AvSigVersion','OsVer','OsBuildLab','Census_OSVersion']
newcolumnnames = []

for colname in versions:
    data = full_features[colname].str.split(r"\.|-",expand=True) # Split if '.' or '-'
    for i in range(data.shape[1]):
        newcolumnname = "%s_%d" % (colname, i+1)
        newcolumnnames.append(newcolumnname)
        full_features[newcolumnname] = data[i]

In [96]:
full_features[newcolumnnames].head(10)

,EngineVersion_1,EngineVersion_2,EngineVersion_3,EngineVersion_4,AppVersion_1,AppVersion_2,AppVersion_3,AppVersion_4,AvSigVersion_1,AvSigVersion_2,AvSigVersion_3,AvSigVersion_4,OsVer_1,OsVer_2,OsVer_3,OsVer_4,OsBuildLab_1,OsBuildLab_2,OsBuildLab_3,OsBuildLab_4,OsBuildLab_5,OsBuildLab_6,Census_OSVersion_1,Census_OSVersion_2,Census_OSVersion_3,Census_OSVersion_4
MachineIdentifier,,,,,,,,,,,,,,,,,,,,,,,,,,
0000028988387b115f69f31a3bf04f09,1,1,15100,1,4,18,1807,18075,1,273,1735,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,165
000007535c3f730efa9ea0b7ef1bd645,1,1,14600,4,4,13,17134,1,1,263,48,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,1
000007905a28d863f6d0d597892cd692,1,1,15100,1,4,18,1807,18075,1,273,1341,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,165
00000b11598a75ea8ba1beea8459149f,1,1,15100,1,4,18,1807,18075,1,273,1527,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,228
000014a5f00daa18e76b81417eeb99fc,1,1,15100,1,4,18,1807,18075,1,273,1379,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,191
000016191b897145d069102325cab760,1,1,15100,1,4,18,1807,18075,1,273,1094,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,165
0000161e8abf8d8b89c5ab8787fd712b,1,1,15100,1,4,18,1807,18075,1,273,845,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,165
000019515bc8f95851aff6de873405e8,1,1,15100,1,4,18,1807,18075,1,273,1393,0,10,0,0,0,14393,0,amd64fre,rs1_release,160715,1616,10,0,14393,0
00001a027a0ab970c408182df8484fce,1,1,15200,1,4,18,1807,18075,1,275,988,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,254


In [97]:
#colname = "OsBuildLab_4"
#print (full_features[colname].value_counts())
#print (colname, full_features[colname].value_counts().keys())

In [98]:
# After splitting the columns, the only values we need to remap are OsBuildLab_3 and OsBuildLab_4
# Other values are already numeric

# OsBuildLab_3 Index(['amd64fre', 'x86fre', 'arm64fre'], dtype='object')

colname = "OsBuildLab_3"
oldvalues = ['amd64fre', 'x86fre', 'arm64fre']
newvalues = [1,3,2]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['amd64fre' 'arm64fre' 'x86fre']
New values [1 2 3]


In [99]:
# OsBuildLab_4 Index(['rs4_release', 'rs3_release_svc_escrow', 'rs3_release', 'rs2_release', 'rs1_release', 'th2_release_sec', 'th1', 'winblue_ltsb_escrow', 'th2_release', 'rs1_release_inmarket', 'winblue_ltsb', 'win7sp1_ldr', 'rs3_release_svc', 'rs1_release_1', 'win7sp1_ldr_escrow', 'rs1_release_sec', 'th1_st1', 'rs5_release', 'rs1_release_inmarket_aim', 'rs3_release_svc_escrow_im', 'th2_release_inmarket', 'rs_prerelease', 'rs_prerelease_flt', 'win7sp1_gdr', 'winblue_gdr', 'th1_escrow', 'win7_gdr', 'winblue_r4', 'rs1_release_inmarket_rim', 'rs1_release_d', 'winblue_r9', 'winblue_r5', 'win7_rtm', 'win7sp1_rtm', 'winblue_r7', 'winblue_r3', 'winblue_r8', 'rs5_release_sigma', 'win7_ldr', 'rs5_release_sigma_dev', 'rs_xbox', 'rs5_release_edge', 'winblue_rtm', 'win7sp1_rc', 'rs3_release_svc_sec', 'rs_onecore_base_cobalt', 'rs6_prerelease', 'rs_onecore_sigma_grfx_dev', 'rs_onecore_stack_per1', 'rs5_release_sign', 'rs_shell']

colname = "OsBuildLab_4"
oldvalues = ['rs4_release', 'rs3_release_svc_escrow', 'rs3_release', 'rs2_release', 'rs1_release', 'th2_release_sec', 'th1', 'winblue_ltsb_escrow', 'th2_release', 'rs1_release_inmarket', 'winblue_ltsb', 'win7sp1_ldr', 'rs3_release_svc', 'rs1_release_1', 'win7sp1_ldr_escrow', 'rs1_release_sec', 'th1_st1', 'rs5_release', 'rs1_release_inmarket_aim', 'rs3_release_svc_escrow_im', 'th2_release_inmarket', 'rs_prerelease', 'rs_prerelease_flt', 'win7sp1_gdr', 'winblue_gdr', 'th1_escrow', 'win7_gdr', 'winblue_r4', 'rs1_release_inmarket_rim', 'rs1_release_d', 'winblue_r9', 'winblue_r5', 'win7_rtm', 'win7sp1_rtm', 'winblue_r7', 'winblue_r3', 'winblue_r8', 'rs5_release_sigma', 'win7_ldr', 'rs5_release_sigma_dev', 'rs_xbox', 'rs5_release_edge', 'winblue_rtm', 'win7sp1_rc', 'rs3_release_svc_sec', 'rs_onecore_base_cobalt', 'rs6_prerelease', 'rs_onecore_sigma_grfx_dev', 'rs_onecore_stack_per1', 'rs5_release_sign', 'rs_shell']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues)

Previous values ['rs4_release' 'rs1_release' 'rs3_release_svc_escrow' 'win7sp1_gdr'
 'rs3_release' 'winblue_ltsb_escrow' 'th1' 'rs1_release_inmarket'
 'rs2_release' 'th2_release' 'winblue_ltsb' 'th2_release_sec'
 'rs3_release_svc_escrow_im' 'th1_st1' 'th2_release_inmarket'
 'rs1_release_sec' 'rs1_release_1' 'win7sp1_ldr' 'win7sp1_ldr_escrow'
 'rs3_release_svc' 'rs1_release_inmarket_aim' 'rs5_release'
 'rs_prerelease_flt' 'rs1_release_inmarket_rim' 'winblue_r7'
 'rs_prerelease' 'win7_gdr' 'rs5_release_sigma' 'winblue_gdr' 'winblue_r4'
 'win7sp1_rtm' 'rs1_release_d' 'th1_escrow' 'rs5_release_edge'
 'winblue_r8' 'win7_ldr' 'winblue_r5' 'win7_rtm' 'winblue_r9' 'winblue_r3'
 'rs3_release_svc_sec' 'rs5_release_sigma_dev' 'win7sp1_rc' 'rs_shell'
 'rs_onecore_stack_per1' 'rs_onecore_sigma_grfx_dev' 'winblue_rtm'
 'rs_onecore_base_cobalt' 'rs_xbox' 'rs6_prerelease' 'rs5_release_sign']
New values [ 1  5  2 24  3  8  7 10  4  9 11  6 20 17 21 16 14 12 15 13 19 18 23 29
 35 22 27 38 25 28 34 30 26

In [100]:
versions = ['EngineVersion','AppVersion','AvSigVersion','OsVer','OsBuildLab','Census_OSVersion']

full_features = full_features.drop(versions, axis=1)

In [111]:
for colname in full_features.columns:
    if full_features[colname].dtypes.name not in ["int8","int16","int32"]:
        full_features[colname] = pd.to_numeric(full_features[colname], errors='coerce')
        topvalue = full_features[colname].value_counts().idxmax()
        full_features[colname].fillna(topvalue, inplace=True)

In [112]:
full_features.head(10)

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_OSEdition,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightsDisabled,Census_FlightRing,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,EngineVersion_1,EngineVersion_2,EngineVersion_3,EngineVersion_4,AppVersion_1,AppVersion_2,AppVersion_3,AppVersion_4,AvSigVersion_1,AvSigVersion_2,AvSigVersion_3,AvSigVersion_4,OsVer_1,OsVer_2,OsVer_3,OsVer_4,OsBuildLab_1,OsBuildLab_2,OsBuildLab_3,OsBuildLab_4,OsBuildLab_5,OsBuildLab_6,Census_OSVersion_1,Census_OSVersion_2,Census_OSVersion_3,Census_OSVersion_4
MachineIdentifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0000028988387b115f69f31a3bf04f09,0,7.0,0,53447.0,1.0,1.0,1,29,128035,18,35.0,171,10,1,256,504,55,1.0,0,0,0,137.0,4,1.0,1.0,1,1,2668.0,9124.0,4.0,5.0,2341.0,476940.0,1,299451.0,0,4096.0,18.9,1440.0,900.0,2,2,1,26.0,119,2,0,1,1,0.0,1,628.0,36144.0,0,0.0,0,0,0.0,0,10,1,1,15100,1,4,18,1807,18075,1,273.0,1735,0,10,0,0,0,17134.0,1.0,1,1,180410.0,1804.0,10,0,17134,165
000007535c3f730efa9ea0b7ef1bd645,0,7.0,0,53447.0,1.0,1.0,1,93,1482,18,119.0,64,10,1,256,504,55,1.0,0,0,0,137.0,4,1.0,1.0,2,1,2668.0,91656.0,4.0,5.0,2405.0,476940.0,1,102385.0,0,4096.0,13.9,1366.0,768.0,1,2,2,8.0,31,2,0,3,1,0.0,2,628.0,57858.0,0,0.0,0,0,0.0,0,8,1,1,14600,4,4,13,17134,1,1,263.0,48,0,10,0,0,0,17134.0,1.0,1,1,180410.0,1804.0,10,0,17134,1
000007905a28d863f6d0d597892cd692,0,7.0,0,53447.0,1.0,1.0,1,86,153579,18,64.0,49,10,1,768,504,52,1.0,0,0,0,137.0,6,1.0,1.0,1,1,4909.0,317701.0,4.0,5.0,1972.0,114473.0,2,113907.0,0,4096.0,21.5,1920.0,1080.0,2,1,1,7.0,30,1,0,1,4,0.0,1,142.0,52682.0,0,0.0,0,0,0.0,0,3,1,1,15100,1,4,18,1807,18075,1,273.0,1341,0,10,0,0,0,17134.0,1.0,1,1,180410.0,1804.0,10,0,17134,165
00000b11598a75ea8ba1beea8459149f,0,7.0,0,53447.0,1.0,1.0,1,88,20710,0,117.0,115,10,1,256,504,55,1.0,0,0,0,137.0,4,1.0,1.0,1,1,1443.0,275890.0,4.0,5.0,2273.0,238475.0,3,227116.0,0,4096.0,18.5,1366.0,768.0,2,2,1,17.0,64,1,0,1,4,0.0,1,355.0,20050.0,0,0.0,0,0,0.0,0,3,1,1,15100,1,4,18,1807,18075,1,273.0,1527,0,10,0,0,0,17134.0,1.0,1,1,180410.0,1804.0,10,0,17134,228
000014a5f00daa18e76b81417eeb99fc,0,7.0,0,53447.0,1.0,1.0,1,18,37376,0,277.0,75,10,1,768,504,52,1.0,0,0,0,137.0,6,1.0,1.0,2,1,1443.0,331929.0,4.0,5.0,2500.0,476940.0,1,101900.0,0,6144.0,14.0,1366.0,768.0,1,1,3,8.0,31,1,0,1,1,0.0,1,355.0,19844.0,0,0.0,0,0,0.0,0,1,1,1,15100,1,4,18,1807,18075,1,273.0,1379,0,10,0,0,0,17134.0,1.0,1,1,180410.0,1804.0,10,0,17134,191
000016191b897145d069102325cab760,0,7.0,0,53447.0,1.0,1.0,1,97,13598,27,126.0,124,10,1,256,504,55,1.0,0,0,0,137.0,6,1.0,1.0,1,1,3799.0,340727.0,2.0,5.0,4324.0,114473.0,2,113671.0,0,8192.0,21.5,1920.0,1080.0,2,2,1,18.0,72,1,0,1,1,0.0,1,93.0,51039.0,0,0.0,0,0,0.0,0,15,1,1,15100,1,4,18,1807,18075,1,273.0,1094,0,10,0,0,0,17134.0,1.0,1,1,180410.0,1804.0,10,

In [113]:
# Let's see some details of the loaded data
full_features.describe()

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_OSEdition,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightsDisabled,Census_FlightRing,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,EngineVersion_1,EngineVersion_2,EngineVersion_3,EngineVersion_4,AppVersion_1,AppVersion_2,AppVersion_3,AppVersion_4,AvSigVersion_1,AvSigVersion_2,AvSigVersion_3,AvSigVersion_4,OsVer_1,OsVer_2,OsVer_3,OsVer_4,OsBuildLab_1,OsBuildLab_2,OsBuildLab_3,OsBuildLab_4,OsBuildLab_5,OsBuildLab_6,Census_OSVersion_1,Census_OSVersion_2,Census_OSVersion_3,Census_OSVersion_4
count,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8921483.0,8921483.0,8.921483e+06,8.921483e+06,8921483.0,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8921483.0,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06
mean,7.509962e-06,6.845893e+00,1.733378e-02,4.786277e+04,1.325452e+00,1.020882e+00,9.879711e-01,1.080490e+02,7.830235e+04,1.719621e+01,1.696641e+02,1.228161e+02,1.315615e+01,1.182901e+00,5.751534e+02,4.800748e+02,5.263172e+01,9.458434e-01,2.891896e-05,2.590377e-04,4.350174e-04,1.266448e+02,4.851504e+00,9.788018e-01,1.301312e+01,2.199818e+00,1.001620e+00,2.224959e+03,2.399955e+05,3.989743e+00,4.533721e+00,2.372781e+03,3.073530e+06,1.418861e+00,3.752959e+05,7.718728e-02,6.097033e+03,1.666998e+01,1.546759e+03,8.968883e+02,1.370257e+00,1.978409e+00,2.944290e+00,1.456586e+01,6.046534e+01,1.883548e+00,5.452008e-04,1.145706e+00,1.596226e+00,9.863831e-06,1.014833e+00,3.975212e+02,3.302931e+04,4.860229e-01,7.026859e-03,1.255431e-01,3.807091e-02,5.696004e-02,2.739330e-01,7.615417e+00,1.0,1.0,1.507467e+04,1.297016e+00,4.0,1.586526e+01,5.645272e+03,1.409716e+04,9.999924e-01,2.723574e+02,9.347381e+02,0.0,9.870695e+00,7.593054e-02,1.082477e-02,6.018058e-04,1.571997e+04,1.421471e+03,1.182901e+00,3.077122e+00,1.765025e+05,1.777148e+03,9.999991e+00,5.156093e-06,1.583483e+04,9.730486e+02
std,2.740421e-03,1.024237e+00,1.305118e-01,1.400839e+04,5.2227

In [114]:
full_features['UacLuaenable'].unique()

array([1.0000000e+00, 0.0000000e+00, 4.8000000e+01, 3.0000000e+00,
       2.0000000e+00, 6.3570620e+06, 4.9000000e+01, 1.6777216e+07,
       5.0000000e+00, 2.5500000e+02, 7.7988840e+06])

In [166]:
# Shuffle the data
np.random.seed(0)

shuffle = np.random.permutation(np.arange(full_features.shape[0]))[:200000]
indexes = full_features.index[shuffle]

random_full_features = full_features.loc[indexes,:]
random_full_labels = full_labels[shuffle]

print (random_full_features.shape, random_full_labels.shape)

(200000, 86) (200000,)


In [167]:
# Probably better to delete
# AvSigVersion
# Census_ChassisTypeName
# Census_InternalPrimaryDiagonalDisplaySizeInInches
# Census_OSEdition
# Census_PrimaryDiskTotalCapacity
# Census_SystemVolumeTotalCapacity
# Census_TotalPhysicalRAM
# IsBeta
# AutoSampleOptIn
# LocaleEnglishNameIdentifier
# Census_IsFlightsDisabled
# Census_FirmwareManufacturerIdentifier
# Census_FirmwareVersionIdentifier
# Census_IsVirtualDevice
# Census_IsAlwaysOnAlwaysConnectedCapable

#random_full_features = random_full_features.drop(['AvSigVersion_1','AvSigVersion_2','AvSigVersion_3','AvSigVersion_4'], axis=1)
#random_full_features = random_full_features.drop(['Census_InternalPrimaryDiagonalDisplaySizeInInches'], axis=1)
#random_full_features = random_full_features.drop(['Census_OSEdition'], axis=1)
#random_full_features = random_full_features.drop(['Census_PrimaryDiskTotalCapacity'], axis=1)
#random_full_features = random_full_features.drop(['Census_SystemVolumeTotalCapacity'], axis=1)
#random_full_features = random_full_features.drop(['Census_TotalPhysicalRAM'], axis=1)
#random_full_features = random_full_features.drop(['IsBeta'], axis=1)
#random_full_features = random_full_features.drop(['AutoSampleOptIn'], axis=1)
#random_full_features = random_full_features.drop(['LocaleEnglishNameIdentifier'], axis=1)
#random_full_features = random_full_features.drop(['Census_IsFlightsDisabled'], axis=1)
#random_full_features = random_full_features.drop(['Census_FirmwareManufacturerIdentifier'], axis=1)
#random_full_features = random_full_features.drop(['Census_FirmwareVersionIdentifier'], axis=1)
#random_full_features = random_full_features.drop(['Census_IsVirtualDevice'], axis=1)
#random_full_features = random_full_features.drop(['Census_IsAlwaysOnAlwaysConnectedCapable'], axis=1)


In [168]:
train_features, test_features, train_labels, test_labels = \
    train_test_split(random_full_features.values, random_full_labels.values, train_size=0.90)

print (train_features.shape, test_features.shape, train_labels.shape, test_labels.shape)

(180000, 85) (20000, 85) (180000,) (20000,)


In [169]:
scaler = StandardScaler()
scaler.fit(train_features)

normalized_train_features = scaler.transform(train_features)
normalized_test_features = scaler.transform(test_features)

In [170]:
algorithms = {
    #"KNeighborsClassifier": KNeighborsClassifier(n_neighbors=50),
    #"DecisionTree": tree.DecisionTreeClassifier(max_depth=50),
    #"RandomForest": ske.RandomForestClassifier(n_estimators=200),
    #"GradientBoosting": ske.GradientBoostingClassifier(n_estimators=200),
    "AdaBoost": ske.AdaBoostClassifier(n_estimators=100),
    #"GNB": GaussianNB(),
    #"MultinomialNB": MultinomialNB()
}

In [171]:
results = {}
print('Testing algorithms using normalized original dataset...\n')
for algo in algorithms:
    clf = algorithms[algo]
    clf.fit(normalized_train_features, train_labels)
    score = clf.score(normalized_test_features, test_labels)
    print("%s : %f %%" % (algo, score*100))
    results[algo] = score
    
winner = max(results, key=results.get)
print(f'Winning algorithm is {winner} with a {results[winner]*100}% accuracy')

Testing algorithms using normalized original dataset...

AdaBoost : 62.265000 %
Winning algorithm is AdaBoost with a 62.265% accuracy


In [172]:
# AdaBoost : 62.360000 % - All columns
# AdaBoost : 62.400000 % - drop(['AvSigVersion_1','AvSigVersion_2','AvSigVersion_3','AvSigVersion_4'], axis=1)
# AdaBoost : 62.320000 % - drop(['Census_InternalPrimaryDiagonalDisplaySizeInInches'], axis=1)
# AdaBoost : 62.185000 % - drop(['Census_OSEdition'], axis=1)
# AdaBoost : 62.030000 % - drop(['Census_PrimaryDiskTotalCapacity'], axis=1)
# AdaBoost : 62.215000 % - drop(['Census_SystemVolumeTotalCapacity'], axis=1)
# AdaBoost : 62.265000 % - drop(['Census_TotalPhysicalRAM'], axis=1)
# drop(['IsBeta'], axis=1)
# drop(['AutoSampleOptIn'], axis=1)
# drop(['LocaleEnglishNameIdentifier'], axis=1)
# drop(['Census_IsFlightsDisabled'], axis=1)
# drop(['Census_FirmwareManufacturerIdentifier'], axis=1)
# drop(['Census_FirmwareVersionIdentifier'], axis=1)
# drop(['Census_IsVirtualDevice'], axis=1)
# drop(['Census_IsAlwaysOnAlwaysConnectedCapable'], axis=1)

In [175]:
cols = [
    ['AvSigVersion_1','AvSigVersion_2','AvSigVersion_3','AvSigVersion_4'],
    ['Census_InternalPrimaryDiagonalDisplaySizeInInches'],
    ['Census_OSEdition'],
    ['Census_PrimaryDiskTotalCapacity'],
    ['Census_SystemVolumeTotalCapacity'],
    ['Census_TotalPhysicalRAM'],
    ['IsBeta'],
    ['AutoSampleOptIn'],
    ['LocaleEnglishNameIdentifier'],
    ['Census_IsFlightsDisabled'],
    ['Census_FirmwareManufacturerIdentifier'],
    ['Census_FirmwareVersionIdentifier'],
    ['Census_IsVirtualDevice'],
    ['Census_IsAlwaysOnAlwaysConnectedCapable']
]

for c in cols:
    random_full_features = full_features.loc[indexes,:]
    random_full_labels = full_labels[shuffle]

    random_full_features = random_full_features.drop(c, axis=1)
    
    train_features, test_features, train_labels, test_labels = \
    train_test_split(random_full_features.values, random_full_labels.values, train_size=0.90)
    scaler = StandardScaler()
    scaler.fit(train_features)
    normalized_train_features = scaler.transform(train_features)
    normalized_test_features = scaler.transform(test_features)
    
    clf = ske.AdaBoostClassifier(n_estimators=100)
    clf = algorithms[algo]
    clf.fit(normalized_train_features, train_labels)
    score = clf.score(normalized_test_features, test_labels)
    
    print (c, train_features.shape, test_features.shape, train_labels.shape, test_labels.shape, "AdaBoostClassifier", score*100)


['AvSigVersion_1', 'AvSigVersion_2', 'AvSigVersion_3', 'AvSigVersion_4'] (180000, 82) (20000, 82) (180000,) (20000,) AdaBoostClassifier 62.09
['Census_InternalPrimaryDiagonalDisplaySizeInInches'] (180000, 85) (20000, 85) (180000,) (20000,) AdaBoostClassifier 62.255
['Census_OSEdition'] (180000, 85) (20000, 85) (180000,) (20000,) AdaBoostClassifier 62.015
['Census_PrimaryDiskTotalCapacity'] (180000, 85) (20000, 85) (180000,) (20000,) AdaBoostClassifier 61.91
['Census_SystemVolumeTotalCapacity'] (180000, 85) (20000, 85) (180000,) (20000,) AdaBoostClassifier 61.475
['Census_TotalPhysicalRAM'] (180000, 85) (20000, 85) (180000,) (20000,) AdaBoostClassifier 61.78
['IsBeta'] (180000, 85) (20000, 85) (180000,) (20000,) AdaBoostClassifier 61.47
['AutoSampleOptIn'] (180000, 85) (20000, 85) (180000,) (20000,) AdaBoostClassifier 62.214999999999996
['LocaleEnglishNameIdentifier'] (180000, 85) (20000, 85) (180000,) (20000,) AdaBoostClassifier 62.025
['Census_IsFlightsDisabled'] (180000, 85) (20000, 